In [40]:
import webbrowser
import csv
import re
import numpy as np
import pandas as pd
import datetime
import requests
import os
import urllib
import urllib.request
import time


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException

from konlpy.tag import Okt
# from nltk.tokenize import word_tokenize 
from collections import Counter

In [41]:
def init(keyword, driver):
    url = 'https://tiktogram.com/location/'  #검색 홈페이지
    driver.get(url+keyword) #드라이버에 전달   #버튼 객체 가져오기
    return driver

In [42]:
def csv_save(save_data):
    data=pd.DataFrame(save_data)
    now = datetime.datetime.now()
    nowDatetime = str(now.strftime('%Y%m%d%H%M%S'))
    data.to_csv(nowDatetime+".csv", sep=',', index=False, header=None, encoding='utf-8-sig')

In [43]:
def get_Post_count(driver):
    return len(driver.find_elements_by_xpath("/html/body/div[3]/div[3]/ul/li"))

In [44]:
def get_PostId(driver,n):
    return driver.find_element_by_xpath("/html/body/div[3]/div[3]/ul/li["+str(n+1)+"]/div/div[2]/div[1]/a").text

In [45]:
def get_PostContent(driver,n):
    return driver.find_element_by_xpath("/html/body/div[3]/div[3]/ul/li["+str(n+1)+"]/div/div[2]/div[2]").text

In [46]:
def get_PostLocation(driver,n):
    return driver.find_element_by_xpath("/html/body/div[3]/div[3]/ul/li["+str(n+1)+"]/div/div[2]/div[3]/span/a").text

In [47]:
def clean_text(text):
#     cleaned_text = re.sub('[a-zA-z]','',text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"\♥\♡\ㅋ\ㅠ\ㅜ\ㄱ\ㅎ\ㄲ\ㅡ]','',text)
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)  # 이모티콘 문자 제거
#     number = re.compile("[^0-9]")
    hangul = re.compile('[^ ㄱ-ㅣ가-힣0-9]+')
    cleaned_text = RE_EMOJI.sub('', cleaned_text)
#     cleaned_text = number.sub('', cleaned_text)
    cleaned_text = hangul.sub('', cleaned_text)
    return cleaned_text

In [48]:
def word_count(word_list,n):
    count=Counter(word_list)
    return count.most_common(n)

In [49]:
def load_Post(driver,ALL_POST_COUNT,GOAL_POST_COUNT):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while ALL_POST_COUNT<GOAL_POST_COUNT:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)                                              
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        ALL_POST_COUNT=get_Post_count(driver)
        print("현재 로딩된 게시물 수 : " + str(ALL_POST_COUNT))
    return ALL_POST_COUNT

In [50]:
if __name__ == "__main__":
    driver = webdriver.Chrome('chromedriver')
    url='https://www.picuki.com/location/ulsan-korea/5771181'
    driver.get(url) #드라이버에 전달   #버튼 객체 가져오기
    SCROLL_PAUSE_TIME = 5
    ALL_POST_COUNT = 0
    GOAL_POST_COUNT = 10000
    WORD_COUNT = 300
    okt = Okt()
    stop_words='아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영'
    stop_words=stop_words.split(' ')

    try:
        print("호출 완료 게시물 수 : "+str(load_Post(driver,ALL_POST_COUNT,GOAL_POST_COUNT)))
        print("목표 완료 게시물 수 : "+str(GOAL_POST_COUNT))
    except NoSuchElementException:
        pass
    except ElementNotInteractableException:
        pass
    except TimeoutException:
        pass

    result = []
    for i in range(get_Post_count(driver)):
        try:
#             print(get_PostId(driver,i))
#             print(clean_text(get_PostContent(driver,i)))
            word_nouns = okt.nouns(clean_text(get_PostContent(driver,i)))
#             print(get_PostLocation(driver,i))
            for w in word_nouns:
                if w not in stop_words: 
                    result.append(w)
        except NoSuchElementException:
            pass
        except ElementNotInteractableException:
            pass
        except TimeoutException:
            pass
    for v in word_count(result,WORD_COUNT):
        print(v)

현재 로딩된 게시물 수 : 195
현재 로딩된 게시물 수 : 327
현재 로딩된 게시물 수 : 459
현재 로딩된 게시물 수 : 591
현재 로딩된 게시물 수 : 657
현재 로딩된 게시물 수 : 723
현재 로딩된 게시물 수 : 789
현재 로딩된 게시물 수 : 921
현재 로딩된 게시물 수 : 1053
현재 로딩된 게시물 수 : 1185
현재 로딩된 게시물 수 : 1317
현재 로딩된 게시물 수 : 1449
현재 로딩된 게시물 수 : 1515
현재 로딩된 게시물 수 : 1647
현재 로딩된 게시물 수 : 1779


KeyboardInterrupt: 

In [51]:
print(result)

['하루', '말', '사람', '하루', '사람', '거울', '잠', '밤', '여름방학', '끝', '어린이', '여름방학', '연장', '예정', '코로나', '사회', '거리', '두기', '서로', '탓', '나부', '종식', '내일', '뭐', '가축', '친구', '보람', '찬', '주말', '년뒤', '머리', '더', '딩', '인생', '타이밍', '뭐', '타이밍', '타이밍', '와중', '귀요미', '유니', '행복', '인생', '모토', '일상', '하루', '셀', '스타', '그램', '셀피', '데', '일리', '선팔', '맞팔', '팔로우', '소통', '울산', '부산', '대형', '미장원', '원장', '욤', '어쩌', '자꾸', '강아지', '신호', '플레이', '보우', '울', '계속', '포착', '바람', '사투', '랍니', '바람', '기계로', '모든', '호기심', '천국', '시야', '다리', '털', '보이', '그랴', '다리', '털', '지지', '조만간', '털', '정리', '한번', '데', '벌써', '땀', '집', '울산', '여행', '중', '가장', '장면', '랄', '마', '시일', '휴가', '연', '쮸륵', '푸릇', '박스', '샤인', '머', '스캣', '용', '체리', '망고', '푸릇', '박스', '골드', '키위', '그린', '키위', '과일', '가득', '푸릇', '박스', '일요일', '잠', '내일', '카페', '엽서', '항상', '카메라', '가방', '카페', '찍기', '요', '엄두', '안나', '요즘', '요', '기분', '바람', '기분', '바람', '손', '발', '상큼', '여행', '분', '손발', '술시', '할인', '깡패', '무슈', '꼬리', '귀', '정', '끝', '더', '깡패', '테디', '브로콜리', '캔디', '주둥이', '달라', '무슈', '진행', '꼬리', '동글동글', '보이

In [ ]:
for v in word_count(result,WORD_COUNT):
    print(v)

In [52]:
csv_save(word_count(result,WORD_COUNT))